In [31]:
! pip install selenium
! pip install html_table_parser

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for beautifulsoup4==4.4.1 from https://files.pythonhosted.org/packages/cf/69/9abfdab06490af5e0233bcebe3f617ec128486d94ea987ad4f77b9332eef/beautifulsoup4-4.4.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/81.4 kB ? eta -:--:--
   --------------- ------------------------ 30.7/81.4 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 81.4/81.4 kB 1.5 MB/s eta 0:00:00
  Created wheel for html_table_parser: filename=html_table_parser-0.1.0-py3-none-any.whl size=4638 sha256=2f599eac9f28c620288fd47ba00512a4bafaa8e8464d32c88e500e5168094cc8
  Stored in directory: c:\users\melid\appdata\local\pip\cache\wheels\3f\61\3e\205c3503f7c244357c1ba105368ba70095a71a075624689850
Successfully built html_table_parser
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.12.2
    Uninstalling beau

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [2]:
driver = webdriver.Chrome()

In [3]:
url = "https://statiz.sporki.com/stats/?m=main&m2=all"
driver.get(url)

In [4]:
driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/section/div[1]/div/button[4]').click()

# 팀 옵션 드롭다운 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/button').click()
# 두산 베어스 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/ul/li[5]').click()

# 수비 > 내야 선택
driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/section/div[6]/ul/li[3]/a').click()

In [5]:
from urllib.request import urlopen

import collections
collections.Callable = collections.abc.Callable

from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import pandas as pd

In [6]:
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

In [7]:
temp = soup.find_all('table')
p = parser.make2d(temp[0])    
for i in range(len(p)):
    print(i, p[i])
print(len(p))

0 ['Rank', 'Name', 'Team', 'Sort▼', 'G', 'GS', 'IP', 'ifH', 'FC', 'PO', 'Ass', 'E', 'BBO', 'BBO%', 'F%', 'RF9', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도']
1 ['Rank', 'Name', 'Team', '수비 관련 득점 기여도-종합 RAA', 'G', 'GS', 'IP', 'ifH', 'FC', 'PO', 'Ass', 'E', 'BBO', 'BBO%', 'F%', 'RF9', 'Range RAA', 'Err RAA', 'DP RAA', 'Bt RAA', '종합 RAA▼', '/144']
2 ['1', '허경민', '24  3B', '5.02', '41', '40', '322.1', '15', '1', '21', '66', '3', '47', '59.6', '96.7', '2.43', '4.62', '0.15', '0.23', '0.02', '5.02', '17.21']
3 ['2', '강승호', '24  2B', '2.98', '40', '40', '325.0', '12', '0', '80', '105', '9', '107', '80.4', '95.4', '5.12', '3.05', '-0.29', '0.23', '', '2.98', '9.99']
4 ['3', '박준영', '24  SS', '2.47', '33', '30', '261.0', '14', '0', '47', '75', '2', '60', '73.3', '98.4', '4.21', '3.11', '0.43', '-1.07', '', '2.47', '10.45']
5 ['4', '양석환', '24  1B', '1.63', '40', '40', '339.0', '13', '0', '308', '21', '1', '170', '91.8', '99.7', '8.73', '0.68', '0.

In [12]:
num = (len(p) // 10)-1

start_num = num
for i in range(num*2+1, 1, -1):
    n = start_num * 10 + (i)
    p.pop(n)
    if i % 2 == 0:
        start_num -= 1

In [16]:
data=pd.DataFrame(p)
data=data[2:]
data.columns = ["Rank", "Name", "Position","Total RAA2","G", "GS","IP", "ifH","FC","PO","Ass","E","BBO","BBO%","F%","RF9","Range RAA", " Err RAA", "DP RAA","Bt RAA","Total RAA", "/144"]
data.drop(columns=['Total RAA2'], inplace=True)
data['Position'].replace({"24  ":""}, regex=True, inplace=True)
data['Team']="Doosan"
print(data)

   Rank Name Position   G  GS     IP ifH FC   PO  Ass  ...    BBO%     F%  \
2     1  허경민       3B  41  40  322.1  15  1   21   66  ...    59.6   96.7   
3     2  강승호       2B  40  40  325.0  12  0   80  105  ...    80.4   95.4   
4     3  박준영       SS  33  30  261.0  14  0   47   75  ...    73.3   98.4   
5     4  양석환       1B  40  40  339.0  13  0  308   21  ...    91.8   99.7   
6     5  이유찬       2B   8   0   13.0   1  0    2    8  ...    80.0  100.0   
7     6  이유찬       3B   1   1    8.0   0  0    1    2  ...          100.0   
8     7  전민재       SS  13   6   64.0   0  0   15   17  ...    93.3   97.0   
9     8  강승호       1B  11   3   39.0   5  0   29    5  ...    79.2  100.0   
10    9  전민재       3B   9   2   30.0   3  0    3    4  ...  -200.0  100.0   
11   10  서예일       3B   2   0    3.0   0  0    0    1  ...   100.0  100.0   
12   11  이유찬       SS   1   0    1.0   0  0    0    1  ...          100.0   
13   12  박준영       3B   1   0    1.0   0  0    0    0  ...                  

In [17]:
data.to_csv("Doosan_field_IF.csv")